In [1]:
!pip install snowflake-ml-python --upgrade

# MONAI Lung CT Registration - Model Training

This notebook trains a **LocalNet** deep learning model for 3D CT lung registration using the MONAI framework.

## Architecture Overview
- **Model**: LocalNet (spatial_dims=3, in_channels=2, out_channels=3)
- **Loss**: Multi-scale local NCC (Normalized Cross-Correlation)
- **Regularization**: Bending energy on deformation field
- **Framework**: MONAI + PyTorch

## Model Registration
After training, the model is registered to **Snowflake Model Registry** using a `CustomModel` class that supports `run_batch()` for distributed GPU inference on compute pools.

## Step 1: Initialize Snowflake Session

Import core libraries and establish connection to Snowflake's active session. This session provides access to:
- Snowflake stages for data storage
- SQL execution capabilities
- File I/O operations

In [4]:
# Core Python libraries for data manipulation and UI
import pandas as pd     # For tabular data handling
# Snowflake-specific imports
from snowflake.snowpark.context import get_active_session

# Establish connection to the active Snowflake session
# This provides access to stages, warehouses, and compute resources
session = get_active_session()
session.use_database('SF_CLINICAL_DB')
session.use_schema('UTILS')

## Step 2: Configure Snowflake ML Jobs

Snowflake ML Jobs provide a simpler alternative to Ray for distributed GPU training:

- **No cluster management**: Jobs automatically provision GPU resources
- **Automatic dependency handling**: Container Runtime includes ML libraries
- **Native Snowflake integration**: Direct access to stages and tables

The `@remote` decorator submits functions to run on GPU compute pools.

In [ ]:
# ============================================================================
# SNOWFLAKE ML JOBS SETUP
# ============================================================================
# Using Snowflake ML Jobs instead of Ray for distributed GPU training.
# Jobs automatically handle compute pool allocation and dependency management.

# Standard Python libraries
import pandas as pd
import logging
import tempfile
import os

# Deep learning frameworks
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Snowflake integrations
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.files import SnowflakeFile
from snowflake.ml.jobs import remote  # ML Jobs API

session = get_active_session()

# ============================================================================
# CONFIGURATION
# ============================================================================

# Compute pool for GPU training jobs
TRAINING_COMPUTE_POOL = "GPU_ML_M_POOL"  # Must have GPU instances (e.g., GPU_NV_S)
PAYLOAD_STAGE = "@SF_CLINICAL_DB.UTILS.RESULTS_STG/job_payloads"

print("✅ Snowflake ML Jobs configured")
print(f"   Compute Pool: {TRAINING_COMPUTE_POOL}")
print(f"   Payload Stage: {PAYLOAD_STAGE}")


In [7]:
!pip install nibabel monai

## Step 3: Data Exploration - Visualize Medical Images

Before training, let's inspect our data! This cell provides an interactive viewer for NIfTI (Neuroimaging Informatics Technology Initiative) files stored in Snowflake stages.

**NIfTI Format**: Standard medical imaging format storing 3D volumetric data (like CT or MRI scans).

**Interactive Slider**: Navigate through the scan's depth (z-axis) to view different slices of the lung.

In [8]:
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import tempfile
import os
from IPython.display import display

from snowflake.snowpark.files import SnowflakeFile
from snowflake.snowpark.context import get_active_session

from monai.networks.nets import LocalNet
from monai.networks.blocks import Warp
from monai.losses import GlobalMutualInformationLoss, BendingEnergyLoss
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd, Resized
)
from monai.data import Dataset, DataLoader


def load_nifti_from_stage(stage_path):
    session = get_active_session()
    clean_path = stage_path if stage_path.startswith("@") else f"@{stage_path}"
    
    try:
        with SnowflakeFile.open(clean_path, 'rb') as f:
            with tempfile.NamedTemporaryFile(suffix=".nii.gz", delete=False) as tmp:
                tmp.write(f.read())
                tmp_name = tmp.name
        
        img = nib.load(tmp_name)
        data = img.get_fdata()
        os.unlink(tmp_name)
        return data
        
    except Exception as e:
        print(f"Error reading {clean_path}: {e}")
        return None


def visualize_nifti_slices(stage_path, num_slices=5):
    print(f"Viewing: {stage_path}")
    
    vol_data = load_nifti_from_stage(stage_path)
    if vol_data is None:
        return

    x, y, z = vol_data.shape
    print(f"Volume Shape: {vol_data.shape}")

    slice_indices = [int(i * (z - 1) / (num_slices - 1)) for i in range(num_slices)]
    
    fig, axes = plt.subplots(1, num_slices, figsize=(4 * num_slices, 4))
    
    for ax, idx in zip(axes, slice_indices):
        slice_data = np.rot90(vol_data[:, :, idx])
        ax.imshow(slice_data, cmap='gray')
        ax.axis('off')
        ax.set_title(f"Slice {idx}")
    
    plt.tight_layout()
    display(fig)
    plt.close(fig)


def visualize_nifti_slice(stage_path, slice_idx=None):
    vol_data = load_nifti_from_stage(stage_path)
    if vol_data is None:
        return

    x, y, z = vol_data.shape
    if slice_idx is None:
        slice_idx = z // 2
    
    slice_data = np.rot90(vol_data[:, :, slice_idx])
    
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(slice_data, cmap='gray')
    ax.axis('off')
    ax.set_title(f"Slice {slice_idx} of {z}")
    display(fig)
    plt.close(fig)


sample_path = "sf_clinical_db.utils.monai_medical_images_stg/scansExp/case_001_exp.nii.gz"
visualize_nifti_slices(sample_path, num_slices=5)

## Step 4: Load Paired Image Paths (Metadata Only)

**Key Insight**: We DON'T load the actual image data yet - just the file paths!

This cell:
1. Lists all NIfTI files in the Snowflake stage using SQL
2. Pairs "fixed" (expiration) and "moving" (inspiration) scans
3. Returns a list of dictionaries containing file paths

**Why paths only?** Loading all medical images into memory would consume GB/TB of RAM. Instead, we'll load images just-in-time during training.

In [12]:
def load_paired_paths(stage_location="@SF_CLINICAL_DB.UTILS.MONAI_MEDICAL_IMAGES_STG"):
    """
    Load paired CT scan paths for medical image registration.
    
    This function performs METADATA OPERATIONS ONLY - no actual image data is loaded.
    It identifies pairs of:
    - Fixed image (expiration CT)
    - Moving image (inspiration CT)
    - Fixed label (lung mask for expiration)
    - Moving label (lung mask for inspiration)
    
    Args:
        stage_location (str): Snowflake stage containing medical images
    
    Returns:
        list: List of dictionaries with file paths for each case
    """
    session = get_active_session()
    
    # ========================================================================
    # STEP 1: List all NIfTI files using SQL (fast, metadata-only operation)
    # ========================================================================
    df_files = session.sql(
        f"LIST {stage_location} PATTERN = '.*.nii.gz'"
    ).select('"name"').to_pandas()
    
    df_files.rename(columns={'"name"': 'name'})
    df_files["name"] = "UTILS." + df_files["name"]
  
    print(df_files)
    
    # ========================================================================
    # STEP 2: Categorize files by type using regex patterns
    # ========================================================================
    fixed_imgs = df_files[df_files['name'].str.contains("scans.*_exp", regex=True)]
    moving_imgs = df_files[df_files['name'].str.contains("scans.*_insp", regex=True)]
    fixed_masks = df_files[df_files['name'].str.contains("lungMasks.*_exp", regex=True)]
    moving_masks = df_files[df_files['name'].str.contains("lungMasks.*_insp", regex=True)]
    
    pairs = []
    
    # ========================================================================
    # STEP 3: Match files into training pairs using case IDs
    # ========================================================================
    for _, row in fixed_imgs.iterrows():
        f_path = row['name']
        filename = f_path.split('/')[-1] 
        case_id = filename.split('_exp')[0] 
        
        m_path = moving_imgs[moving_imgs['name'].str.contains(f"{case_id}_insp")].iloc[0]['name']
        fl_path = fixed_masks[fixed_masks['name'].str.contains(f"{case_id}_exp")].iloc[0]['name']
        ml_path = moving_masks[moving_masks['name'].str.contains(f"{case_id}_insp")].iloc[0]['name']
        
        pairs.append({
            "case_id": case_id,
            "fixed_path": f_path,
            "moving_path": m_path,
            "fixed_label_path": fl_path,
            "moving_label_path": ml_path
        })
        
    print(f"✅ Paired {len(pairs)} cases (paths only, no binary data loaded)")
    return pairs


# ============================================================================
# EXECUTE: Load paired file paths
# ============================================================================
paired_data = load_paired_paths()


## Step 5: Define Custom Dataset with Just-In-Time Loading

This cell creates a PyTorch `Dataset` that:
1. **Loads images on-demand** - Downloads from Snowflake stage only when needed
2. **Applies preprocessing** - Normalizes intensities, resizes volumes
3. **Handles labels correctly** - Preserves binary masks without intensity scaling

**Why custom dataset?** Standard datasets can't handle Snowflake stage I/O. This custom implementation downloads each file to a temporary location, processes it, then cleans up.

In [13]:
# ============================================================================
# HELPER FUNCTIONS FOR SNOWFLAKE STAGE I/O
# ============================================================================

def read_file_from_stage(stage_path):
    """
    Read binary file content from Snowflake stage.
    
    Args:
        stage_path (str): Path to file in stage (with or without @ prefix)
    
    Returns:
        bytes: Raw file content
    """
    session = get_active_session()
    # Ensure path has @ prefix for Snowflake stage access
    clean_path = stage_path if stage_path.startswith("@") else f"@{stage_path}"
    
    with SnowflakeFile.open(clean_path, 'rb') as f:
        return f.read()


# ============================================================================
# CUSTOM PYTORCH DATASET WITH SNOWFLAKE INTEGRATION
# ============================================================================

class SnowflakeStageDataset(Dataset):
    """
    PyTorch Dataset for medical images stored in Snowflake stages.
    
    Key Features:
    1. **Just-In-Time Loading**: Downloads images from Snowflake only when needed
    2. **Automatic Cleanup**: Deletes temporary files after processing
    3. **MONAI Integration**: Applies medical imaging transformations
    4. **Separate Transforms**: Different pipelines for images vs. labels
    
    This approach avoids loading all GB/TB of medical data into memory at once.
    
    Args:
        data_dicts (list): List of dicts with keys like 'fixed_path', 'moving_path', etc.
        transform_img (Callable): MONAI transforms for images (includes normalization)
        transform_lbl (Callable): MONAI transforms for labels (NO normalization)
    """
    def __init__(self, data_dicts, transform_img=None, transform_lbl=None):
        self.data_dicts = data_dicts
        self.transform_img = transform_img
        self.transform_lbl = transform_lbl

    def __len__(self):
        """Return number of image pairs in dataset."""
        return len(self.data_dicts)

    def __getitem__(self, index):
        """
        Load and preprocess a single training sample.
        
        This method:
        1. Downloads 4 files from Snowflake (fixed/moving images + labels)
        2. Saves them to temporary locations
        3. Applies MONAI transformations
        4. Cleans up temporary files
        5. Returns preprocessed tensors
        
        Args:
            index (int): Dataset index
        
        Returns:
            dict: Dictionary with 'fixed', 'moving', 'fixed_label', 'moving_label' tensors
        """
        item = self.data_dicts[index]
        
        # ====================================================================
        # STEP 1: Download files from Snowflake to temporary locations
        # ====================================================================
        temp_files = {}
        for key in ["fixed", "moving", "fixed_label", "moving_label"]:
            path = item[f"{key}_path"]
            binary = read_file_from_stage(path)
            
            # Create temporary file with .nii.gz extension (required by nibabel)
            tf = tempfile.NamedTemporaryFile(suffix=".nii.gz", delete=False)
            tf.write(binary)
            tf.close()
            temp_files[key] = tf.name
        
        # ====================================================================
        # STEP 2: Prepare data dictionary for MONAI transforms
        # ====================================================================
        # MONAI expects file paths as input to LoadImaged transform
        data = {
            "fixed": temp_files["fixed"],
            "moving": temp_files["moving"],
            "fixed_label": temp_files["fixed_label"],
            "moving_label": temp_files["moving_label"],
        }
        
        # ====================================================================
        # STEP 3: Apply MONAI transformations (load, normalize, resize, augment)
        # ====================================================================
        if self.transform_img:
            data = self.transform_img(data)
        
        # ====================================================================
        # STEP 4: Clean up temporary files to free disk space
        # ====================================================================
        for fpath in temp_files.values():
            if os.path.exists(fpath):
                os.unlink(fpath)
        
        return data

## Step 6: Define Training Function

This is the core training loop submitted as a **Snowflake ML Job**:

1. **Defines Transforms** - Preprocessing pipeline for medical images
2. **Creates Data Loaders** - Wraps our custom dataset for batch iteration
3. **Builds Model** - LocalNet architecture for deformation field prediction
4. **Training Loop** - Iterates through epochs, computing loss and updating weights
5. **Validation** - Monitors Dice score on held-out data
6. **Model Checkpointing** - Saves best model to Snowflake stage

The `@remote` decorator submits this function to run on a GPU compute pool.

In [25]:
# ============================================================================
# MONAI MEDICAL IMAGING IMPORTS
# ============================================================================

from monai.networks.nets import LocalNet
from monai.networks.blocks import Warp
from monai.losses import GlobalMutualInformationLoss, BendingEnergyLoss
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    ScaleIntensityRanged,
    Resized,
    LoadImage,
    EnsureChannelFirst,
    ScaleIntensityRange,
    Resize,
    RandAffined,
    RandGaussianNoised,
    RandGaussianSmoothd
)
from monai.data import Dataset, DataLoader

CT_MIN_HU = -1000
CT_MAX_HU = 500

CONFIG = {
    "spatial_size": (96, 96, 104),
    "num_channel_initial": 32,
    "batch_size": 2,
    "learning_rate": 1e-4,
    "max_epochs": 25,
    "reg_weight": 1.0,
    "save_interval": 10,
}


# ============================================================================
# DISTRIBUTED TRAINING FUNCTION (GPU-ACCELERATED)
# ============================================================================

@remote(
    compute_pool=TRAINING_COMPUTE_POOL,
    stage_name=PAYLOAD_STAGE,
    pip_requirements=["monai", "nibabel", "pytorch-ignite"],
    external_access_integrations=["ALLOW_ALL_EAI"]
)
def train_registration_model(
    train_files, 
    val_files=None,
    config=None,
    save_path="@SF_CLINICAL_DB.UTILS.RESULTS_STG"
):
    import torch
    import torch.optim as optim
    import numpy as np
    import tempfile
    import os
    
    from snowflake.snowpark.context import get_active_session
    from snowflake.snowpark.files import SnowflakeFile
    from monai.networks.nets import LocalNet
    from monai.networks.blocks import Warp
    from monai.losses import GlobalMutualInformationLoss, BendingEnergyLoss
    from monai.transforms import (
        Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityRanged, Resized,
        RandAffined, RandGaussianNoised, RandGaussianSmoothd
    )
    from monai.data import Dataset as MonaiDataset, DataLoader
    
    CT_MIN_HU = -1000
    CT_MAX_HU = 500
    
    if config is None:
        config = {
            "spatial_size": (96, 96, 104),
            "num_channel_initial": 32,
            "batch_size": 2,
            "learning_rate": 1e-4,
            "max_epochs": 25,
            "reg_weight": 1.0,
            "save_interval": 10,
        }
    
    print(f"🚀 Starting training with config: {config}")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"📍 Using device: {device}")
    
    def read_file_from_stage(stage_path):
        clean_path = stage_path if stage_path.startswith("@") else f"@{stage_path}"
        with SnowflakeFile.open(clean_path, 'rb') as f:
            return f.read()
    
    class SnowflakeStageDataset(MonaiDataset):
        def __init__(self, data_dicts, transform_img=None):
            self.data_dicts = data_dicts
            self.transform_img = transform_img

        def __len__(self):
            return len(self.data_dicts)

        def __getitem__(self, index):
            item = self.data_dicts[index]
            temp_files = {}
            for key in ["fixed", "moving", "fixed_label", "moving_label"]:
                path = item[f"{key}_path"]
                binary = read_file_from_stage(path)
                tf = tempfile.NamedTemporaryFile(suffix=".nii.gz", delete=False)
                tf.write(binary)
                tf.close()
                temp_files[key] = tf.name
            
            data = {
                "fixed": temp_files["fixed"],
                "moving": temp_files["moving"],
                "fixed_label": temp_files["fixed_label"],
                "moving_label": temp_files["moving_label"],
            }
            
            if self.transform_img:
                data = self.transform_img(data)
            
            for fpath in temp_files.values():
                if os.path.exists(fpath):
                    os.unlink(fpath)
            
            return data
    
    train_img_transforms = Compose([
        LoadImaged(keys=["fixed", "moving", "fixed_label", "moving_label"]),
        EnsureChannelFirstd(keys=["fixed", "moving", "fixed_label", "moving_label"]),
        ScaleIntensityRanged(
            keys=["fixed", "moving"],
            a_min=CT_MIN_HU,
            a_max=CT_MAX_HU,
            b_min=0.0,
            b_max=1.0,
            clip=True
        ),
        Resized(
            keys=["fixed", "moving"], 
            spatial_size=config["spatial_size"],
            mode="trilinear"
        ),
        Resized(
            keys=["fixed_label", "moving_label"], 
            spatial_size=config["spatial_size"], 
            mode="nearest"
        ),
    ])
    
    val_transforms = train_img_transforms
    
    train_ds = SnowflakeStageDataset(train_files, transform_img=train_img_transforms)
    train_loader = DataLoader(
        train_ds, 
        batch_size=config["batch_size"], 
        shuffle=True, 
        num_workers=0
    )
    
    val_loader = None
    if val_files:
        val_ds = SnowflakeStageDataset(val_files, transform_img=val_transforms)
        val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0)
    
    model = LocalNet(
        spatial_dims=3,
        in_channels=2,
        out_channels=3,
        num_channel_initial=config["num_channel_initial"],
        extract_levels=[3],
        out_activation=None,
        out_kernel_initializer="zeros"
    ).to(device)
    
    warp_layer = Warp().to(device)
    
    loss_sim = GlobalMutualInformationLoss()
    loss_reg = BendingEnergyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )
    
    best_val_dice = 0.0
    training_history = []
    
    def save_model_to_stage(model, stage_path):
        session = get_active_session()
        filename = stage_path.split('/')[-1]
        local_path = f"/tmp/{filename}"
        torch.save(model.state_dict(), local_path)
        stage_dir = '/'.join(stage_path.split('/')[:-1])
        try:
            session.file.put(local_path, stage_dir, overwrite=True, auto_compress=False)
            print(f"✅ Saved to {stage_path}")
        except Exception as e:
            print(f"❌ Failed to save: {e}")
        finally:
            if os.path.exists(local_path):
                os.unlink(local_path)
    
    for epoch in range(config["max_epochs"]):
        model.train()
        epoch_loss = 0
        epoch_sim_loss = 0
        epoch_reg_loss = 0
        step = 0
        
        for batch_data in train_loader:
            step += 1
            optimizer.zero_grad()
            
            fixed = batch_data["fixed"].to(device)
            moving = batch_data["moving"].to(device)
            
            input_tensor = torch.cat((moving, fixed), dim=1)
            ddf = model(input_tensor)
            pred_image = warp_layer(moving, ddf)
            
            sim_loss = loss_sim(pred_image, fixed)
            reg_loss = loss_reg(ddf)
            total_loss = sim_loss + config["reg_weight"] * reg_loss
            
            total_loss.backward()
            optimizer.step()
            
            epoch_loss += total_loss.item()
            epoch_sim_loss += sim_loss.item()
            epoch_reg_loss += reg_loss.item()
        
        avg_loss = epoch_loss / step
        avg_sim = epoch_sim_loss / step
        avg_reg = epoch_reg_loss / step
        
        print(f"✅ Epoch {epoch + 1}/{config['max_epochs']}")
        print(f"   Loss: {avg_loss:.4f} (Sim: {avg_sim:.4f}, Reg: {avg_reg:.4f})")
        
        avg_val_dice = None
        if val_loader:
            model.eval()
            val_dice_scores = []
            
            with torch.no_grad():
                for val_batch in val_loader:
                    fixed = val_batch["fixed"].to(device)
                    moving = val_batch["moving"].to(device)
                    fixed_lbl = val_batch["fixed_label"].to(device)
                    moving_lbl = val_batch["moving_label"].to(device)
                    
                    input_tensor = torch.cat((moving, fixed), dim=1)
                    ddf = model(input_tensor)
                    pred_label = warp_layer(moving_lbl, ddf)
                    
                    intersection = (pred_label * fixed_lbl).sum()
                    dice = (2.0 * intersection + 1e-5) / (pred_label.sum() + fixed_lbl.sum() + 1e-5)
                    val_dice_scores.append(dice.item())
            
            avg_val_dice = np.mean(val_dice_scores)
            print(f"   Val Dice: {avg_val_dice:.4f}")
            
            scheduler.step(avg_loss)
            
            if avg_val_dice > best_val_dice:
                best_val_dice = avg_val_dice
                print(f"   🏆 New best model! Dice: {best_val_dice:.4f}")
                save_model_to_stage(model, f"{save_path}/best_model.pth")
        
        training_history.append({
            "epoch": epoch + 1,
            "train_loss": avg_loss,
            "train_sim": avg_sim,
            "train_reg": avg_reg,
            "val_dice": avg_val_dice
        })
        
        if (epoch + 1) % config["save_interval"] == 0:
            checkpoint_path = f"{save_path}/checkpoint_epoch_{epoch+1}.pth"
            save_model_to_stage(model, checkpoint_path)
            print(f"   💾 Checkpoint saved: {checkpoint_path}")
    
    final_path = f"{save_path}/final_model.pth"
    save_model_to_stage(model, final_path)
    print(f"🏁 Training complete! Final model: {final_path}")
    
    return {
        "final_model_path": final_path,
        "best_model_path": f"{save_path}/best_model.pth",
        "best_val_dice": best_val_dice,
        "training_history": training_history
    }

## Step 7: Prepare Training and Validation Sets

Now we split our dataset into:
- **Training Set (80%)** - Used for model optimization
- **Validation Set (20%)** - Used for monitoring performance and early stopping

This split helps prevent overfitting and ensures our model generalizes to unseen data.

In [26]:
train_files_list = []

for item in paired_data:
    train_files_list.append({
        "fixed_path": item["fixed_path"],
        "moving_path": item["moving_path"],
        "fixed_label_path": item["fixed_label_path"],
        "moving_label_path": item["moving_label_path"]
    })

split_idx = int(0.8 * len(train_files_list))

train_split = train_files_list[:split_idx]
val_split = train_files_list[split_idx:]

print(f"📊 Training: {len(train_split)} pairs, Validation: {len(val_split)} pairs")

## Step 8: Launch Training

This cell defines hyperparameters and launches training on the GPU compute pool via Snowflake ML Jobs.

**Expected Training Time**: ~10-30 minutes depending on dataset size and GPU.

In [27]:
session.use_database("SF_CLINICAL_DB")
session.use_schema("UTILS")

# ============================================================================
# TRAINING CONFIGURATION & HYPERPARAMETERS
# ============================================================================

# ============================================================================
# CT WINDOWING PARAMETERS (Hounsfield Units)
# ============================================================================
# These define the CT intensity range we're interested in for lung imaging
CT_MIN_HU = -1000  # Air (lowest HU in lungs)
CT_MAX_HU = 500    # Soft tissue/bone (upper limit for lung CT)

# ============================================================================
# MODEL & TRAINING HYPERPARAMETERS
# ============================================================================
CONFIG = {
    # Image size after preprocessing (downsampled for memory efficiency)
    "spatial_size": (96, 96, 104),  # (Height, Width, Depth)
    
    # Network architecture parameter (controls model capacity)
    "num_channel_initial": 32,  # Number of feature channels in first layer
    
    # Training parameters
    "batch_size": 2,            # Number of image pairs per batch (limited by GPU memory)
    "learning_rate": 1e-4,      # Adam optimizer learning rate
    "max_epochs": 25,           # Total training iterations through dataset
    
    # Loss function weighting
    "reg_weight": 1.0,          # Weight for regularization (smoothness penalty)
                                # Lower = more flexible deformations
                                # Higher = smoother but potentially less accurate
    
    # Checkpointing
    "save_interval": 10,        # Save model checkpoint every N epochs
}


# ============================================================================
# LAUNCH TRAINING AS SNOWFLAKE ML JOB
# ============================================================================

# Submit training job to GPU compute pool
# The @remote decorator handles job submission automatically
print("🚀 Submitting training job to compute pool...")

training_job = train_registration_model(
    train_files=train_split,       # Training image pairs
    val_files=val_split,           # Validation image pairs
    config=CONFIG,                 # Hyperparameters defined above
    save_path="@SF_CLINICAL_DB.UTILS.RESULTS_STG"  # Snowflake stage for checkpoints
)

print(f"📋 Job ID: {training_job.id}")
print(f"📊 Status: {training_job.status}")

# Monitor job progress
import time
while training_job.status not in ["DONE", "FAILED"]:
    print(f"⏳ Job status: {training_job.status}")
    time.sleep(30)  # Check every 30 seconds

# Get final result
if training_job.status == "DONE":
    training_result = training_job.result()
    st.success(f"✅ Training Complete! Best Dice: {training_result['best_val_dice']:.4f}")
    st.json(training_result)
else:
    print("❌ Training failed. Logs:")
    print(training_job.get_logs())


## Step 9: Verify Saved Model Checkpoints

Let's verify that our model was successfully saved to the Snowflake stage during training.

In [28]:
%%sql -r dataframe_1
ls @results_stg;

## Step 10: Register Model in Snowflake Model Registry (run_batch Compatible)

Now we register our trained model using a **Custom Model Class** that enables:
- **`run_batch()` support** - Distributed batch inference on GPU compute pools
- **Automatic dependency management** - No manual pip installs on workers
- **Version Control** - Track different model versions over time
- **Simplified deployment** - Just call `model.run_batch()` for inference

The Custom Model wraps our MONAI LocalNet and handles:
1. Loading images from Snowflake stages
2. Preprocessing (CT windowing, resizing)
3. Running inference
4. Calculating metrics (Dice score)
5. Saving results back to stages

In [ ]:
import torch
import io
import os
import tempfile
import numpy as np
import pandas as pd
import nibabel as nib

from snowflake.snowpark.context import get_active_session
from snowflake.ml.model import custom_model
from snowflake.ml.registry import Registry
from monai.networks.nets import LocalNet
from monai.networks.blocks import Warp
from monai.transforms import (
    Compose, LoadImage, EnsureChannelFirst, ScaleIntensityRange, Resize
)

session = get_active_session()

CT_MIN_HU = -1000
CT_MAX_HU = 500
SPATIAL_SIZE = (96, 96, 104)


class LungCTRegistrationModel(custom_model.CustomModel):
    def __init__(self, context: custom_model.ModelContext) -> None:
        super().__init__(context)
        self._model = None
        self._warp_layer = None
        self._device = None
    
    def _load_model(self):
        if self._model is None:
            from monai.networks.nets import LocalNet
            from monai.networks.blocks import Warp
            
            model_path = self.context.path("model_weights")
            
            self._model = LocalNet(
                spatial_dims=3,
                in_channels=2,
                out_channels=3,
                num_channel_initial=32,
                extract_levels=[3],
                out_activation=None,
                out_kernel_initializer="zeros"
            )
            
            state_dict = torch.load(model_path, map_location='cpu')
            self._model.load_state_dict(state_dict)
            self._model.eval()
            
            self._device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self._model = self._model.to(self._device)
            self._warp_layer = Warp().to(self._device)
    
    def _get_transforms(self):
        from monai.transforms import (
            Compose, LoadImage, EnsureChannelFirst, ScaleIntensityRange, Resize
        )
        
        preprocess_image = Compose([
            LoadImage(image_only=True),
            EnsureChannelFirst(),
            ScaleIntensityRange(
                a_min=-1000, a_max=500,
                b_min=0.0, b_max=1.0, clip=True
            ),
            Resize(spatial_size=(96, 96, 104), mode="trilinear"),
        ])
        
        preprocess_label = Compose([
            LoadImage(image_only=True),
            EnsureChannelFirst(),
            Resize(spatial_size=(96, 96, 104), mode="nearest"),
        ])
        
        return preprocess_image, preprocess_label
    
    def _preprocess_from_bytes(self, data: bytes, is_label: bool = False) -> torch.Tensor:
        with tempfile.NamedTemporaryFile(suffix=".nii.gz", delete=False) as tmp:
            tmp.write(data)
            tmp_path = tmp.name
        
        try:
            preprocess_image, preprocess_label = self._get_transforms()
            if is_label:
                tensor = preprocess_label(tmp_path)
            else:
                tensor = preprocess_image(tmp_path)
            return tensor.unsqueeze(0).to(self._device)
        finally:
            if os.path.exists(tmp_path):
                os.unlink(tmp_path)
    
    @custom_model.inference_api
    def predict(self, input_df: pd.DataFrame) -> pd.DataFrame:
        self._load_model()
        results = []
        
        for idx, row in input_df.iterrows():
            case_id = row.get('CASE_ID', row.get('case_id', f'case_{idx}'))
            
            try:
                fixed_img = self._preprocess_from_bytes(
                    row.get('FIXED_PATH', row.get('fixed_path')), is_label=False)
                moving_img = self._preprocess_from_bytes(
                    row.get('MOVING_PATH', row.get('moving_path')), is_label=False)
                fixed_lbl = self._preprocess_from_bytes(
                    row.get('FIXED_LABEL_PATH', row.get('fixed_label_path')), is_label=True)
                moving_lbl = self._preprocess_from_bytes(
                    row.get('MOVING_LABEL_PATH', row.get('moving_label_path')), is_label=True)
                
                with torch.no_grad():
                    input_tensor = torch.cat((moving_img, fixed_img), dim=1)
                    ddf = self._model(input_tensor)
                    reg_label = self._warp_layer(moving_lbl, ddf)
                    reg_image = self._warp_layer(moving_img, ddf)
                    
                    intersection = (reg_label * fixed_lbl).sum()
                    dice = (2.0 * intersection + 1e-5) / (reg_label.sum() + fixed_lbl.sum() + 1e-5)
                    
                    ddf_magnitude = torch.sqrt((ddf ** 2).sum(dim=1)).mean()
                
                results.append({
                    "CASE_ID": case_id,
                    "STATUS": "success",
                    "DICE_SCORE": float(dice.item()),
                    "DDF_MAGNITUDE": float(ddf_magnitude.item()),
                    "DEVICE_USED": str(self._device)
                })
                
            except Exception as e:
                results.append({
                    "CASE_ID": case_id,
                    "STATUS": f"failed: {str(e)}",
                    "DICE_SCORE": -1.0,
                    "DDF_MAGNITUDE": -1.0,
                    "DEVICE_USED": "N/A"
                })
        
        return pd.DataFrame(results)


def register_model_for_run_batch(
    model_stage_path="@SF_CLINICAL_DB.UTILS.RESULTS_STG/best_model.pth",
    model_name="LUNG_CT_REGISTRATION",
    version_name="v_batch"
):
    print(f"Registering {model_name} {version_name} for run_batch()...")
    
    raw_stream = session.file.get_stream(model_stage_path)
    with tempfile.NamedTemporaryFile(suffix=".pth", delete=False) as tmp:
        tmp.write(raw_stream.read())
        local_weights_path = tmp.name
    
    try:
        model_context = custom_model.ModelContext(
            model_weights=local_weights_path
        )
        
        my_model = LungCTRegistrationModel(model_context)
        
        registry = Registry(
            session,
            database_name="SF_CLINICAL_DB",
            schema_name="UTILS"
        )
        
        sample_input = pd.DataFrame({
            "CASE_ID": ["test"],
            "FIXED_PATH": [b"dummy"],
            "MOVING_PATH": [b"dummy"],
            "FIXED_LABEL_PATH": [b"dummy"],
            "MOVING_LABEL_PATH": [b"dummy"]
        })
        
        model_ref = registry.log_model(
            model=my_model,
            model_name=model_name,
            version_name=version_name,
            pip_requirements=["monai", "nibabel", "numpy", "pandas", "torch"],
            sample_input_data=sample_input,
            options={
                "cuda_version": "12.3"
            }
        )
        
        print(f"Registered: {model_ref.fully_qualified_model_name}")
        print(f"Ready for run_batch() on GPU compute pools!")
        return model_ref
        
    finally:
        if os.path.exists(local_weights_path):
            os.unlink(local_weights_path)



## Step 11: Execute Model Registration

Run the registration function and verify the model appears in the registry.

In [ ]:
# ============================================================================
# EXECUTE MODEL REGISTRATION
# ============================================================================

# Option 1: Register for run_batch() (RECOMMENDED)
# This enables distributed GPU inference via model.run_batch()
model_ref = register_model_for_run_batch(
    model_stage_path="@SF_CLINICAL_DB.UTILS.RESULTS_STG/best_model.pth",
    model_name="LUNG_CT_REGISTRATION",
    version_name="V_RUN_BATCH_8"
)


# ============================================================================
# VERIFY REGISTRATION SUCCESS
# ============================================================================

registry = Registry(
    session, 
    database_name="SF_CLINICAL_DB", 
    schema_name="UTILS"
)

all_models = registry.show_models()
print("\n📚 All registered models in SF_CLINICAL_DB.UTILS:")
all_models


## Step 12: Test Model Loading & Inference

Verify we can load the registered model and run inference. This confirms the model is ready for deployment!

In [ ]:
from snowflake.ml.registry import Registry


# ============================================================================
# VERIFY REGISTRATION SUCCESS
# ============================================================================

registry = Registry(
    session, 
    database_name="SF_CLINICAL_DB", 
    schema_name="UTILS"
)

all_models = registry.show_models()
print("\n📚 All registered models in SF_CLINICAL_DB.UTILS:")
print(all_models)


# Retrieve the registered model by name and version
loaded = registry.get_model("LUNG_CT_REGISTRATION").version("V_RUN_BATCH_8")
print(f"\n✅ Successfully loaded: {loaded.model_name}")


## Training Complete

### What We Accomplished
1. Configured Snowflake ML Jobs for GPU training
2. Loaded medical images from Snowflake stages
3. Trained LocalNet model for CT registration using MONAI
4. Saved model checkpoints to Snowflake stage
5. Registered model in Snowflake Model Registry with `run_batch()` support
6. Verified inference with test case

### Model Location
- **Checkpoints**: `@SF_CLINICAL_DB.UTILS.RESULTS_STG/best_model.pth`
- **Registry**: `SF_CLINICAL_DB.UTILS.LUNG_CT_REGISTRATION`

**Ready to run batch inference? Proceed to notebook 4!**

In [ ]:
# ============================================================================
# RUN BATCH INFERENCE ON GPU COMPUTE POOL (Test with single case)
# ============================================================================

from snowflake.ml.model import JobSpec, OutputSpec, SaveMode, InputSpec, InputFormat, FileEncoding

registry = Registry(session, database_name="SF_CLINICAL_DB", schema_name="UTILS")
model = registry.get_model("LUNG_CT_REGISTRATION")
mv = model.version("V_RUN_BATCH_8")

input_data = [
    {
        "CASE_ID": "case_001",
        "FIXED_PATH": "@SF_CLINICAL_DB.UTILS.MONAI_MEDICAL_IMAGES_STG/scansExp/case_001_exp.nii.gz",
        "MOVING_PATH": "@SF_CLINICAL_DB.UTILS.MONAI_MEDICAL_IMAGES_STG/scansInsp/case_001_insp.nii.gz",
        "FIXED_LABEL_PATH": "@SF_CLINICAL_DB.UTILS.MONAI_MEDICAL_IMAGES_STG/lungMasksExp/case_001_exp.nii.gz",
        "MOVING_LABEL_PATH": "@SF_CLINICAL_DB.UTILS.MONAI_MEDICAL_IMAGES_STG/lungMasksInsp/case_001_insp.nii.gz"
    }
]

input_df = session.create_dataframe(input_data)

job = mv.run_batch(
    compute_pool=TRAINING_COMPUTE_POOL,
    X=input_df,
    input_spec=InputSpec(
        column_handling={
            "FIXED_PATH": {"input_format": InputFormat.FULL_STAGE_PATH, "convert_to": FileEncoding.RAW_BYTES},
            "MOVING_PATH": {"input_format": InputFormat.FULL_STAGE_PATH, "convert_to": FileEncoding.RAW_BYTES},
            "FIXED_LABEL_PATH": {"input_format": InputFormat.FULL_STAGE_PATH, "convert_to": FileEncoding.RAW_BYTES},
            "MOVING_LABEL_PATH": {"input_format": InputFormat.FULL_STAGE_PATH, "convert_to": FileEncoding.RAW_BYTES}
        }
    ),
    output_spec=OutputSpec(
        stage_location="@SF_CLINICAL_DB.UTILS.RESULTS_STG/inference_out/",
        mode=SaveMode.OVERWRITE
    ),
    job_spec=JobSpec(gpu_requests="1")
)

print("Batch inference job submitted!")
print(f"Job ID: {job.id}")

job.wait()
print("Job completed!")

In [27]:
%%sql -r dataframe_2
ls @SF_CLINICAL_DB.UTILS.RESULTS_STG/inference_out/

In [29]:
%%sql -r dataframe_3
CREATE OR REPLACE FILE FORMAT sf_clinical_db.utils.parquet_ff TYPE = 'parquet';


select $1
from @SF_CLINICAL_DB.UTILS.results_stg/inference_out/3_f7de682d39c24e16b9002dd28c9d8be1_000000_000000-0.parquet
 (file_format => sf_clinical_db.utils.parquet_ff) 